In [2]:
import octomap

In [3]:
print(dir(octomap))

['NullPointerException', 'OcTree', 'OcTreeKey', 'OcTreeNode', '__builtins__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '__test__', '_octree_read', 'iterator_base', 'leaf_bbx_iterator', 'leaf_iterator', 'np', 'tree_iterator']


In [4]:
import pyrealsense2 as rs
import open3d as o3d
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.transform import Rotation as R
from mpl_toolkits.mplot3d import Axes3D
import time

# Collecting frames for next dev

In [5]:
count_of_frames = 20
frames_between_frames_D435 = 15

In [6]:
count_of_frames * frames_between_frames_D435 / 30

10.0

# Iitial configs

In [7]:
# data filenames
d435_filename = '../data/D435.bag'

In [8]:
# config fo D435
cfg_d435 = rs.config()
cfg_d435.enable_device_from_file(d435_filename)
cfg_d435.enable_stream(rs.stream.depth, 848, 480, rs.format.z16, 30)
pipe_d435 = rs.pipeline()

## Collecting frames

In [9]:
d435_frames = []
pipe_d435.start(cfg_d435)
while len(d435_frames) != count_of_frames:
    for i in range(frames_between_frames_D435):
        depth_frame = pipe_d435.wait_for_frames().get_depth_frame()
    d435_frames.append(depth_frame)
pipe_d435.stop()

In [10]:
def count_diff_frames(frames):
    prev_frame = None
    for i, frame in enumerate(frames):
        if prev_frame is not None:
            print("Difference bw {0} and {1} frames is {2:.4f} ms".format(i, i-1, frame.get_timestamp()-prev_frame.get_timestamp()))
        prev_frame = frame

In [11]:
count_diff_frames(d435_frames)

Difference bw 1 and 0 frames is 500.7671 ms
Difference bw 2 and 1 frames is 499.8582 ms
Difference bw 3 and 2 frames is 500.9062 ms
Difference bw 4 and 3 frames is 499.7070 ms
Difference bw 5 and 4 frames is 500.0930 ms
Difference bw 6 and 5 frames is 500.2417 ms
Difference bw 7 and 6 frames is 500.3752 ms
Difference bw 8 and 7 frames is 500.2996 ms
Difference bw 9 and 8 frames is 500.2456 ms
Difference bw 10 and 9 frames is 500.3613 ms
Difference bw 11 and 10 frames is 500.2937 ms
Difference bw 12 and 11 frames is 500.7803 ms
Difference bw 13 and 12 frames is 499.8145 ms
Difference bw 14 and 13 frames is 500.7185 ms
Difference bw 15 and 14 frames is 499.9082 ms
Difference bw 16 and 15 frames is 500.3823 ms
Difference bw 17 and 16 frames is 500.2190 ms
Difference bw 18 and 17 frames is 500.4111 ms
Difference bw 19 and 18 frames is 500.2153 ms


In [12]:
depth_frame = d435_frames[0]
depth_frame

In [13]:
def get_coordinates(depth_frame, make_sampling=True, koef = 2**2, count_sampling = 1):
    """
     TODO
    :param make_sampling:
    :return:
    """
    pc = rs.pointcloud()
    if make_sampling:
        decimate = rs.decimation_filter()
        decimate.set_option(rs.option.filter_magnitude, koef)
        for i in range(count_sampling):
            depth_frame = decimate.process(depth_frame)
        points = pc.calculate(depth_frame).as_points()
    else:
        points = pc.calculate(depth_frame).as_points()

    coordinates = np.ndarray(buffer=points.get_vertices(), dtype=np.float32, shape=(points.size(), 3))
    coordinates = coordinates[coordinates[:, 2] != 0]

#     coordinates = apply_transformation(tm_T265toD435, coordinates)
    return coordinates


In [14]:
pc = get_coordinates(depth_frame, koef = 2 ** 1, count_sampling = 1)

In [15]:
pc.shape

(91959, 3)

# Octomap

In [16]:
import math3d as m3d
import glooey
import imgviz
import numpy as np
import pyglet
import trimesh
import trimesh.transformations as tf
import trimesh.viewer

In [17]:
m3d.Transform(np.zeros(6)).array

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [18]:
def labeled_scene_widget(scene, label):
    vbox = glooey.VBox()
    vbox.add(glooey.Label(text=label, color=(255, 255, 0)), size=0)
    vbox.add(trimesh.viewer.SceneWidget(scene))
    return vbox


In [19]:
def visualize(occupied, resolution, aabb):
    window = pyglet.window.Window( # pyglet http://pyglet.org/
        width=int(1024), height=int(768)
    )

    @window.event
    def on_key_press(symbol, modifiers):
        if modifiers == 0:
            if symbol == pyglet.window.key.Q:
                window.on_close()

    gui = glooey.Gui(window)
    hbox = glooey.VBox()
#     hbox.set_padding(50)


    aabb_min, aabb_max = aabb
    bbox = trimesh.path.creation.box_outline(
        aabb_max - aabb_min,
        tf.translation_matrix((aabb_min + aabb_max) / 2),
    )

    geom = trimesh.voxel.ops.multibox(
        occupied, pitch=resolution, colors=[1., 0, 0, 0.5]
    )
    scene = trimesh.Scene(geometry=[bbox, geom])

    gui.add(trimesh.viewer.SceneWidget(scene))
    pyglet.app.run()

In [20]:
def visualize(occupied, resolution, aabb):
    window = pyglet.window.Window( # pyglet http://pyglet.org/
        width=int(1024), height=int(768)
    )

    @window.event
    def on_key_press(symbol, modifiers):
        if modifiers == 0:
            if symbol == pyglet.window.key.Q:
                window.on_close()

    gui = glooey.Gui(window)
    hbox = glooey.VBox()
#     hbox.set_padding(50)


    aabb_min, aabb_max = aabb
    bbox = trimesh.path.creation.box_outline(
        aabb_max - aabb_min,
        tf.translation_matrix((aabb_min + aabb_max) / 2),
    )

    geom = trimesh.voxel.ops.multibox(
        occupied, pitch=resolution, colors=[1., 0, 0, 0.5]
    )
    scene = trimesh.Scene(geometry=[bbox, geom])

#     gui.add(trimesh.viewer.SceneWidget(scene))
#     pyglet.app.run()
    return scene

In [21]:
resolution = 0.05
octree = octomap.OcTree(resolution)

octree.insertPointCloud(
    pointcloud=np.double(pc),
    origin=np.array([0, 0, 0], dtype=float),
    maxrange=2,
)


occupied, empty = octree.extractPointCloud()

aabb_min = octree.getMetricMin()
aabb_max = octree.getMetricMax()

visualize(
    occupied=occupied, # no :(
    resolution=resolution,
    aabb=(aabb_min, aabb_max),
)


<trimesh.Scene(len(geometry)=2)>

In [20]:
aabb_min

array([-1.4, -1. ,  0. ])

In [21]:
aabb_max

array([0.45, 0.75, 2.  ])

In [21]:
import threading


class StoppableThread(threading.Thread):
    """Thread class with a stop() method. The thread itself has to check
    regularly for the is_executed() condition."""

    def __init__(self,  *args, **kwargs):
        super(StoppableThread, self).__init__(*args, **kwargs)
        self._execution_status = True

    def stop(self):
        self._execution_status = False

    def is_execute(self):
        return self._execution_status


In [22]:
import time
import numpy as np
import glooey
import octomap
import pyglet # http://pyglet.org/
import trimesh
import trimesh.transformations as tf
import trimesh.viewer
# from helpers.custom_threading import StoppableThread


class OctoMapVisualiser:
    def __init__(self, update_time=1/30, resolution=0.01):
        self.update_time = update_time
        self.resolution = resolution
        self.octree = octomap.OcTree(resolution)

        self.point_cloud = None
        self.is_changed = False
        self.window = pyglet.window.Window(width=1024, height=720)

        @self.window.event
        def on_key_press(symbol, modifiers):
            if modifiers == 0:
                if symbol == pyglet.window.key.Q:
                    self.window.on_close()

        self.gui = glooey.Gui(self.window)
        
#         aabb_min = np.array([-1.4, -1. ,  0. ])
#         aabb_max = np.array([0.45, 0.75, 2.  ])
#         initial_bbox = trimesh.path.creation.box_outline(
#             aabb_max - aabb_min,
#             tf.translation_matrix((aabb_min + aabb_max) / 2),
#         )        
        self.scene = visualize(
                    occupied=occupied, # no :(
                    resolution=resolution,
                    aabb=(aabb_min, aabb_max),
                )

#         trimesh.Scene(geometry=[initial_bbox])
        
        self.app_visualiser = None
        self.start_visualiser()

    def __del__(self):
        print("Deleting octomap")
        pyglet.clock.unschedule(func=self.update_visualisation)
        self.app_visualiser.stop()
        self.window.close()


    def update_visualisation(self, dt):
        if self.is_changed:
            if self.point_cloud is not None:
                self.octree.insertPointCloud(
                    pointcloud=np.double(self.point_cloud),
                    origin=np.array([0, 0, 0], dtype=float), # TODO this
                    maxrange=2,
                )

                occupied, _ = self.octree.extractPointCloud()
                aabb_min = self.octree.getMetricMin()
                aabb_max = self.octree.getMetricMax()

                bbox = trimesh.path.creation.box_outline(
                    aabb_max - aabb_min,
                    tf.translation_matrix((aabb_min + aabb_max) / 2),
                )
                geom = trimesh.voxel.ops.multibox(
                    occupied, pitch=self.resolution, colors=[1., 0, 0, 0.5]
                )
                self.scene = trimesh.Scene(geometry=[bbox, geom])                

#                 self.update_hbox(occupied, aabb_min, aabb_max)

            self.is_changed = False
        else:
            print("Didn't changed")
            pass


    def update_points(self, new_pc):
        self.point_cloud = new_pc
        self.is_changed = True

    def init_visualiser(self):
        pyglet.clock.schedule_interval(func=self.update_visualisation, interval=self.update_time)
        self.gui.add(trimesh.viewer.SceneWidget(self.scene))
        pyglet.app.run()

    def start_visualiser(self):
        self.app_visualiser = StoppableThread(target=self.init_visualiser)
        self.app_visualiser.start()

    def stop_visualiser(self):
        self.__del__()



In [1]:
octo_visualiser = OctoMapVisualiser(update_time=2)

NameError: name 'OctoMapVisualiser' is not defined

In [24]:
octo_visualiser.stop_visualiser()

NameError: name 'octo_visualiser' is not defined

In [19]:
%time
resolution = 0.05
octree = octomap.OcTree(resolution)

octree.insertPointCloud(
    pointcloud=np.double(pc),
    origin=np.array([0, 0, 0], dtype=float),
    maxrange=2,
)

occupied, empty = octree.extractPointCloud()
aabb_min = octree.getMetricMin()
aabb_max = octree.getMetricMax()

CPU times: user 4 µs, sys: 2 µs, total: 6 µs
Wall time: 11.9 µs


In [21]:
from mayavi.mlab import *

In [28]:
occupied

array([[-0.19999999, -0.1       ,  0.54999998],
       [-0.40000001,  0.        ,  1.60000005],
       [-0.15      , -0.65000001,  1.9       ],
       ...,
       [ 0.05      ,  0.65      ,  1.85000005],
       [ 0.05      ,  0.70000001,  1.85000005],
       [ 0.05      ,  0.70000001,  1.9       ]])

In [29]:
# itr = octree
# root_size = itr.size()
# op = []
# fp = []
# so = []
# sf = []
# for i in itr:
#     if i.isLeaf():
#         if self.tree.isNodeOccupied(i):
#             so.append(i.getSize() / root_size)
#             op.append(i.getCoordinate())
#         else:
#             if self.view_free:
#                 sf.append(i.getSize() / root_size)
#                 fp.append(i.getCoordinate())
# op = zip(*op)
# fp = zip(*fp)
# points3d(occupied[:,0], occupied[:,1], occupied[:,2], opacity=1.0, mode='cube',
#          color=(0, 0, 1), scale_mode='scalar', scale_factor=root_size)
# if self.view_free:
#     points3d(fp[0], fp[1], fp[2], sf, opacity=0.3, mode='cube',
#              color=(0, 1, 0), scale_mode='scalar', scale_factor=root_size)
show()

In [20]:
octree.write('tree.oct')

TypeError: expected bytes, str found

In [27]:
occupied.shape

(2146, 3)

In [29]:
pc.shape

(91959, 3)

DEBUG        kernelbase.py:252   
*** MESSAGE TYPE:execute_request***
DEBUG        kernelbase.py:253      Content: {'silent': False, 'store_history': True, 'user_expressions': {}, 'allow_stdin': True, 'stop_on_error': True, 'code': 'resolution = 0.05\noctree = octomap.OcTree(resolution)\n\noctree.insertPointCloud(\n    pointcloud=np.double(pc),\n    origin=np.array([0, 0, 0], dtype=float),\n    maxrange=2,\n)\n\n\noccupied, empty = octree.extractPointCloud()\n\naabb_min = octree.getMetricMin()\naabb_max = octree.getMetricMax()\n\nvisualize(\n    occupied=occupied, # no :(\n    resolution=resolution,\n    aabb=(aabb_min, aabb_max),\n)\n'}
   --->
   
DEBUG        kernelbase.py:262   execute_request: {'header': {'date': datetime.datetime(2020, 3, 29, 12, 9, 8, 671000, tzinfo=tzutc()), 'msg_id': '5d1b7b5c-9510-43e6-a133-cf970fd82245', 'msg_type': 'execute_request', 'session': 'c5eec68d-e0d8-4f92-8d14-0c8d4048b22b', 'username': '', 'version': '5.2'}, 'msg_id': '5d1b7b5c-9510-43e6-a133-cf97

Deleting octomap
Deleting octomap


DEBUG         constants.py:137   smoothed executed in 1.7964 seconds.
DEBUG           caching.py:116   `faces_sparse`: 1.46E-03s, `cache.verify`: 1.56E-06s
DEBUG           caching.py:116   `triangles`: 2.06E-03s, `cache.verify`: 1.79E-06s
DEBUG           caching.py:116   `face_angles`: 3.07E-02s, `cache.verify`: 3.79E-06s
DEBUG           caching.py:116   `vertex_normals`: 6.46E-02s, `cache.verify`: 4.86E-06s
DEBUG           caching.py:116   `vertex_degree`: 8.25E-04s, `cache.verify`: 1.69E-05s


Didn't changed


DEBUG           caching.py:116   `nodes_geometry`: 3.43E-05s, `cache.verify`: 2.14E-06s


Didn't changed
Didn't changed
Didn't changed
Didn't changed
Didn't changed
Didn't changed
Didn't changed
Didn't changed
Didn't changed
Didn't changed
Didn't changed


DEBUG        kernelbase.py:566   {'header': {'msg_id': '44f92128-cfcac781411ad7bd7e2e68a4_391', 'msg_type': 'execute_reply', 'username': 'valeryilin', 'session': '44f92128-cfcac781411ad7bd7e2e68a4', 'date': datetime.datetime(2020, 3, 29, 12, 9, 33, 631766, tzinfo=datetime.timezone.utc), 'version': '5.3'}, 'msg_id': '44f92128-cfcac781411ad7bd7e2e68a4_391', 'msg_type': 'execute_reply', 'parent_header': {'date': datetime.datetime(2020, 3, 29, 12, 9, 8, 671000, tzinfo=tzutc()), 'msg_id': '5d1b7b5c-9510-43e6-a133-cf970fd82245', 'msg_type': 'execute_request', 'session': 'c5eec68d-e0d8-4f92-8d14-0c8d4048b22b', 'username': '', 'version': '5.2'}, 'content': {'status': 'ok', 'execution_count': 33, 'user_expressions': {}, 'payload': []}, 'metadata': {'started': datetime.datetime(2020, 3, 29, 12, 9, 8, 685317, tzinfo=datetime.timezone.utc), 'dependencies_met': True, 'engine': '589bcfc5-0a1e-4b0e-b159-0ee2adee0e6f', 'status': 'ok'}, 'tracker': <zmq.sugar.tracker.MessageTracker object at 0x7faebad64